# Pipeline analysis of the Face Perception Dataset

### Load Libraries 

In [ ]:
import os 
import numpy as np
import mne
import matplotlib
from mne.preprocessing import compute_current_source_density
from mne.io import read_raw_fif, concatenate_raws
from sklearn.pipeline import make_pipeline
from mne.decoding import Scaler, Vectorizer, cross_val_multiscore

In [ ]:
matplotlib.use('Qt5Agg')

### Load the data

In [ ]:
# define paths
data_dir = '/Users/christinadelta/datasets/multimodal_bids'
derivatives_dir = os.path.join(data_dir, 'derivatives')
stimuli_dir = os.path.join(data_dir, 'stimuli')
emptyroom_dir = os.path.join(data_dir, 'sub-emptyroom')
task = 'facerecognition'

In [ ]:
runs = 1
raws = []
subject = 1

In [ ]:
for sub in range(subject):
    this_sub = 'sub-{0:02d}'.format(sub+1)
    for run in range(runs):
        this_run = '{0:02d}'.format(run+1)
        this_eeg = os.path.join(data_dir, this_sub, 'ses-meg', 'meg',
                                f'{this_sub}_ses-meg_task-{task}_run-{this_run}_meg.fif')
        
        this_raw = read_raw_fif(this_eeg, preload=True)
        raws.append(this_raw)

In [ ]:
# merge the runs 
# concatinate the 3 raw files
raw = concatenate_raws(raws)
raw

In [ ]:
eeg_raw.info

### Choose the data
This is a MEG recording that also contains EEG data (74 channels). We will only use the EEG data, so get rid of the MEG channels
We will create a **eeg_raw** class/object for our raw data. To keep only the eeg data we will use the ```mne.pick_types()``` function

In [ ]:
eeg_raw = raw.pick_types(eeg=True, meg=False, stim=True, eog=True)

In [ ]:
eeg_raw.plot(duration=5)

### Plot power spectrum

In [ ]:
eeg_raw.plot_psd(fmin=1, fmax=20, n_fft=2**10, spatial_colors=True)

### Filter the raw eeg data

In [ ]:
eeg_filtered = eeg_raw.filter(l_freq=0.1, h_freq=40)

### Run ICA

In [ ]:
ica = mne.preprocessing.ICA(n_components=15, random_state=97, max_iter=800)

In [ ]:
ica.fit(eeg_filtered)
ica.plot_components()

In [ ]:
# blinks
ica.plot_overlay(eeg_filtered, exclude=[0], picks='eeg')

In [ ]:
# exclude bad components
ica.exclude = [1, 6]  # indices chosen based on various plots above

### Make copy of the original raw, apply ICA and visualise both

In [ ]:
orig_raw = eeg_raw.copy()
ica.apply(eeg_filtered)

### Plot the raw and the cleaned data side by side and inspect

In [ ]:
orig_raw.plot(start=0, duration=5, block=False, title= 'before ICA')
eeg_filtered.plot(start=0, duration=5, block=True, title='after ICA')

### Add events

In [ ]:
events = mne.find_events(eeg_raw)
print(events[:5])

In [ ]:
# create events_id dictionary 
event_id = {
    'famous_in': 5,
    'famous_imm': 6, 
    'famous_del': 7,
    'unfam_in': 13,
    'unfam_imm': 14,
    'unfam_del': 15,
    'scram_in': 17,
    'scram_imm': 18,
    'scram_del': 19
}

In [ ]:
# visualise events
events_fig = mne.viz.plot_events(events, event_id=event_id,  sfreq=eeg_raw.info['sfreq'], 
                          first_samp=eeg_raw.first_samp)

In [ ]:
eeg_filtered.plot(duration=10, events=events, event_id=event_id)

### Epoch the data

In [ ]:
tmin = -0.3
tmax = 0.5
baseline = (-0.20, 0) 
reject_criteria = dict(eeg=150e-6)

In [ ]:
epochs = mne.Epochs(eeg_filtered, 
                    events=events, 
                    event_id=event_id, 
                    tmin=tmin, 
                    tmax=tmax,
                    baseline=baseline,
                    reject=reject_criteria, 
                    preload=True)

In [ ]:
epochs.plot(events=events, event_id=event_id)

### Select epochs of interest

#### Equalise the number of trials for each condition

In [ ]:
epochs.equalize_event_counts(['famous_in', 
                              'famous_imm', 
                              'famous_del', 
                              'unfam_in',
                              'unfam_imm', 
                              'unfam_del', 
                              'scram_in', 
                              'scram_imm', 
                              'scram_del'])

In [ ]:
famous_in = epochs['famous_in']
famous_imm = epochs['famous_imm']
famous_del = epochs['famous_del']
unfam_in = epochs['unfam_in']
unfam_imm = epochs['unfam_imm']
unfam_del = epochs['unfam_del']
scram_in = epochs['scram_in']
scram_imm = epochs['scram_imm']
scram_del = epochs['scram_del']

### Run Decoding Analysis
#### Create a classifier